<a href="https://colab.research.google.com/github/bardiaHSZD/ComputerVisionVFX/blob/main/blender_bake.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup
**Make sure to read the instructions carefully!**

If you have other resources used in the Blender project and chose to *make all paths relative*, pack all of them into a zip archive. Alternatively, you can *pack all external file*.

* `blender_version` : Version of blender used to render the scene.
* `blend_file_path` : Path to the blend file after unpacking the zip archive. If blend file is used, this is automatically ignored.
___
* `upload_type` : Select the type of upload method. `gdrive_relative` pulls everything from the folder specified.
* `drive_path` : Path to your blend/zip file relative to the root of your Google Drive if `google_drive` is selected. Must  state the file and its extension (.zip/.blend) **unless** `gdrive_relative` is selected.
* `url_blend` : Specify the URL to the blend/zip file if `url` is selected.
___
* `animation` : Specify whether animation or still image is rendered. If **still image** is used, put the frame number in `start_frame`.
* `start_frame, end_frame` : Specify the start and end frame for animation. You may put same value such as zero for both input to set the default frame in the blend file.
___
* `download_type` : Select the type of download method. `gdrive_direct` enables the frames to be outputted directly to Google Drive (zipping will be disabled).
* `output_name` : Name of the output frames, **do NOT include .blend!** (## for frame number)
* `zip_files` : Archive multiple animation frames automatically into a zip file.
* `drive_output_path` : Path to your frames/zip file in Google Drive.
___
* `gpu_enabled, cpu_enabled` : Toggle GPU and CPU for rendering. CPU might give a slight boost in rendering time but may varies depend on the project.

After you are done, go to Runtime > Run All (Ctrl + F9) and upload your files or have Google Drive authorised below. See the [GitHub repo](https://github.com/syn73/blender-colab) for more info.

In [1]:
!sudo apt update

!sudo apt install ffmpeg

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:2 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [696 B]
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:9 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:10 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [76.0 kB]
Get:11 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:12 http://security.ubuntu.com/ubu

In [2]:
!apt-get install python-opengl -y

!apt install xvfb -y

!pip install pyvirtualdisplay

!pip install piglet


from pyvirtualdisplay import Display
Display().start()

Reading package lists... Done
Building dependency tree       
Reading state information... Done
Suggested packages:
  libgle3
The following NEW packages will be installed:
  python-opengl
0 upgraded, 1 newly installed, 0 to remove and 62 not upgraded.
Need to get 496 kB of archives.
After this operation, 5,416 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 python-opengl all 3.1.0+dfsg-1 [496 kB]
Fetched 496 kB in 1s (571 kB/s)
Selecting previously unselected package python-opengl.
(Reading database ... 155229 files and directories currently installed.)
Preparing to unpack .../python-opengl_3.1.0+dfsg-1_all.deb ...
Unpacking python-opengl (3.1.0+dfsg-1) ...
Setting up python-opengl (3.1.0+dfsg-1) ...
Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following NEW packages will be installed:
  xvfb
0 upgraded, 1 newly installed, 0 to remove and 62 not upgraded.
Need to get 784 kB of 

In [3]:
blender_version = "3.0.0" #@param ["2.79b", "2.80rc3", "2.81a", "2.82a", "2.83.18", "2.90.1", "2.91.2", "2.92.0", "2.93.5", "3.0.0"]
blend_file_path = 'path/to/file.blend' #@param {type: 'string'}
#@markdown ---
upload_type = 'direct' #@param ['direct', 'google_drive', 'url', 'gdrive_relative'] {allow-input: false}
drive_path = 'path/to/blend.zip' #@param {type: 'string'}
url_blend = '' #@param {type: 'string'}
#@markdown ---
animation = True #@param {type: 'boolean'}
start_frame =  1#@param {type: 'integer'}
end_frame =  260#@param {type: 'integer'}
#@markdown ---
download_type = 'direct' #@param ['direct', 'google_drive', 'gdrive_direct'] {allow-input: false}
output_name = 'blender-##' #@param {type: 'string'}
zip_files = True #@param {type: 'boolean'}
drive_output_path = 'blender/output' #@param {type: 'string'}
#@markdown ---
gpu_enabled = True #@param {type:"boolean"}
cpu_enabled = True #@param {type:"boolean"}

In [4]:
import os
import shutil
from google.colab import files, drive
uploaded_filename = ""

upload_type = 'google_drive'

if upload_type == 'google_drive' or upload_type == 'gdrive_relative' or download_type == 'google_drive' or download_type == 'gdrive_direct':
    drive.mount('/drive')

if upload_type == 'direct':
    uploaded = files.upload()
    for fn in uploaded.keys():
        uploaded_filename = fn
elif upload_type == 'url':
    !wget -nc $url_blend
    uploaded_filename = os.path.basename(url_blend)
elif upload_type == 'google_drive':
    #shutil.copy('/content/drive/MyDrive' + drive_path, '.')
    #uploaded_filename = os.path.basename(drive_path)
    uploaded_filename = '/content/drive/MyDrive/out/Camera_setup.blend'


Mounted at /drive


In [5]:
!cat /proc/cpuinfo
!cat /proc/meminfo

processor	: 0
vendor_id	: GenuineIntel
cpu family	: 6
model		: 63
model name	: Intel(R) Xeon(R) CPU @ 2.30GHz
stepping	: 0
microcode	: 0x1
cpu MHz		: 2299.998
cache size	: 46080 KB
physical id	: 0
siblings	: 40
core id		: 0
cpu cores	: 20
apicid		: 0
initial apicid	: 0
fpu		: yes
fpu_exception	: yes
cpuid level	: 13
wp		: yes
flags		: fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mca cmov pat pse36 clflush mmx fxsr sse sse2 ss ht syscall nx pdpe1gb rdtscp lm constant_tsc rep_good nopl xtopology nonstop_tsc cpuid tsc_known_freq pni pclmulqdq ssse3 fma cx16 pcid sse4_1 sse4_2 x2apic movbe popcnt aes xsave avx f16c rdrand hypervisor lahf_lm abm invpcid_single ssbd ibrs ibpb stibp fsgsbase tsc_adjust bmi1 avx2 smep bmi2 erms invpcid xsaveopt arat md_clear arch_capabilities
bugs		: cpu_meltdown spectre_v1 spectre_v2 spec_store_bypass l1tf mds swapgs
bogomips	: 4599.99
clflush size	: 64
cache_alignment	: 64
address sizes	: 46 bits physical, 48 bits virtual
power management:

processor

In [6]:
#Connect Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
blender_url_dict = {'2.79b'   : "https://ftp.nluug.nl/pub/graphics/blender/release/Blender2.79/blender-2.79b-linux-glibc219-x86_64.tar.bz2",
                    '2.80rc3' : "https://ftp.nluug.nl/pub/graphics/blender/release/Blender2.80/blender-2.80rc3-linux-glibc217-x86_64.tar.bz2",
                    '2.81a'   : "https://ftp.nluug.nl/pub/graphics/blender/release/Blender2.81/blender-2.81a-linux-glibc217-x86_64.tar.bz2",
                    '2.82a'   : "https://ftp.nluug.nl/pub/graphics/blender/release/Blender2.82/blender-2.82a-linux64.tar.xz",
                    '2.83.18' : "https://ftp.nluug.nl/pub/graphics/blender/release/Blender2.83/blender-2.83.18-linux-x64.tar.xz",
                    '2.90.1'  : "https://ftp.nluug.nl/pub/graphics/blender/release/Blender2.90/blender-2.90.1-linux64.tar.xz",
                    '2.91.2'  : "https://ftp.nluug.nl/pub/graphics/blender/release/Blender2.91/blender-2.91.2-linux64.tar.xz",
                    '2.92.0'  : "https://ftp.nluug.nl/pub/graphics/blender/release/Blender2.92/blender-2.92.0-linux64.tar.xz",
                    '2.93.5'  : "https://ftp.nluug.nl/pub/graphics/blender/release/Blender2.93/blender-2.93.5-linux-x64.tar.xz",
                    '3.0.0'   : "https://builder.blender.org/download/daily/blender-3.1.0-alpha+master.c4bedeb0186b-linux.x86_64-release.tar.xz"}

blender_url = blender_url_dict[blender_version]
base_url = os.path.basename(blender_url)

!mkdir $blender_version
!wget -nc $blender_url
!tar -xkf $base_url -C ./$blender_version --strip-components=1

--2022-01-26 09:22:20--  https://builder.blender.org/download/daily/blender-3.1.0-alpha+master.c4bedeb0186b-linux.x86_64-release.tar.xz
Resolving builder.blender.org (builder.blender.org)... 82.94.213.213
Connecting to builder.blender.org (builder.blender.org)|82.94.213.213|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 187892784 (179M) [application/x-xz]
Saving to: ‘blender-3.1.0-alpha+master.c4bedeb0186b-linux.x86_64-release.tar.xz’

blender-3.1.0-alpha 100%[===================>] 179.19M   336KB/s    in 6m 19s  

2022-01-26 09:28:40 (484 KB/s) - ‘blender-3.1.0-alpha+master.c4bedeb0186b-linux.x86_64-release.tar.xz’ saved [187892784/187892784]



In [8]:
# Enable GPU rendering (or add custom properties here)
data = "import re\n"+\
    "import bpy\n"+\
    "scene = bpy.context.scene\n"+\
    "scene.cycles.device = 'GPU'\n"+\
    "prefs = bpy.context.preferences\n"+\
    "prefs.addons['cycles'].preferences.get_devices()\n"+\
    "cprefs = prefs.addons['cycles'].preferences\n"+\
    "print(cprefs)\n"+\
    "for compute_device_type in ('CUDA', 'OPENCL', 'NONE'):\n"+\
    "    try:\n"+\
    "        cprefs.compute_device_type = compute_device_type\n"+\
    "        print('Device found:',compute_device_type)\n"+\
    "        break\n"+\
    "    except TypeError:\n"+\
    "        pass\n"+\
    "for device in cprefs.devices:\n"+\
    "    if not re.match('intel', device.name, re.I):\n"+\
    "        print('Activating',device)\n"+\
    "        device.use = "+str(gpu_enabled)+"\n"+\
    "    else:\n"+\
    "        device.use = "+str(cpu_enabled)+"\n"
with open('setgpu.py', 'w') as f:
    f.write(data)

In [9]:
!sudo ./$blender_version/blender -b -noaudio -P '/content/drive/MyDrive/out/Addon_Enable.py'

Blender 3.1.0 Alpha (hash c4bedeb0186b built 2022-01-26 01:08:48)
addon_utils.disable: flip_fluids_addon not disabled
Modules Installed (flip_fluids_addon) from '/content/drive/MyDrive/out/FLIP_Fluids_addon_1.1.0_stable_(20_aug_2021).zip' into '/root/.config/blender/3.1/scripts/addons'
Info: Modules Installed (flip_fluids_addon) from '/content/drive/MyDrive/out/FLIP_Fluids_addon_1.1.0_stable_(20_aug_2021).zip' into '/root/.config/blender/3.1/scripts/addons'
Info: Modules Installed (flip_fluids_addon) from '/content/drive/MyDrive/out/FLIP_Fluids_addon_1.1.0_stable_(20_aug_2021).zip' into '/root/.config/blender/3.1/scripts/addons'
Writing userprefs: '/root/.config/blender/3.1/config/userpref.blend' ok
Info: Preferences saved
Info: Preferences saved

Blender quit


In [19]:
!sudo ./$blender_version/blender --background '/content/drive/MyDrive/out/camera_setup.blend' --python '/content/drive/MyDrive/out/run_simulation_start.py'

Blender 3.1.0 Alpha (hash c4bedeb0186b built 2022-01-26 01:08:48)
Read prefs: /root/.config/blender/3.1/config/userpref.blend
xcb_connection_has_error() returned true
Read blend: /content/drive/MyDrive/out/camera_setup.blend
Info: Successfully reset bake
Info: Successfully reset bake
Exporting simulation data...
Exporting Simulation Meshes:
------------------------------------------------------------
Exporting... 5 / 24 objects 	(Database size: 884 kB)
Exporting... 6 / 24 objects 	(Database size: 884 kB)
Exporting... 11 / 24 objects 	(Database size: 884 kB)
Exporting... 13 / 24 objects 	(Database size: 884 kB)
Exporting... 24 / 24 objects 	(Database size: 884 kB)
Running fluid simulation...
------------------------------------------------------------
Fluid Engine Version 1.1.0 Stable 20-AUG-2021
------------------------------------------------------------
26-Jan-2022 10h28m26s enablePreviewMeshOutput: 0.0771301
26-Jan-2022 10h28m26s setDomainOffset: -7.65874 -0.631022 -2.90887
26-Jan-2

In [ ]:
!sudo ./$blender_version/blender --background '/content/drive/MyDrive/out/camera_setup.blend' --python '/content/drive/MyDrive/out/run_simulation_chunks.py'

Streaming output truncated to the last 5000 lines.
26-Jan-2022 12h03m56s BEGIN       Calculate Surface Curvature
26-Jan-2022 12h03m56s COMPLETE    Save Velocity Field
26-Jan-2022 12h03m56s BEGIN       Apply Force Fields
26-Jan-2022 12h03m57s COMPLETE    Apply Force Fields
26-Jan-2022 12h03m57s BEGIN       Solve Pressure System
26-Jan-2022 12h03m59s COMPLETE    Calculate Surface Curvature
26-Jan-2022 12h04m11s COMPLETE    Solve Pressure System
26-Jan-2022 12h04m11s BEGIN       Constrain Velocity Field
26-Jan-2022 12h04m11s COMPLETE    Constrain Velocity Field
26-Jan-2022 12h04m11s BEGIN       Simulate Diffuse Material
26-Jan-2022 12h04m20s COMPLETE    Simulate Diffuse Material
26-Jan-2022 12h04m20s BEGIN       Update Sheet Seeding
26-Jan-2022 12h04m22s COMPLETE    Update Sheet Seeding
26-Jan-2022 12h04m22s BEGIN       Update Marker Particle Velocities
26-Jan-2022 12h04m22s COMPLETE    Update Marker Particle Velocities
26-Jan-2022 12h04m22s BEGIN       Update Marker Particle Attributes
2

## Disclaimer
The GPU used in Google Colab is specialized for data centres, neural network etc, not rendering 3D scenes. Because the computing power provided are free, the usage limits, idle timeouts and speed of the rendering may varies. [ColabPro](https://colab.research.google.com/signup) is available for those who wanted to have more powerful GPU and longer session for rendering. See the [FAQ](https://research.google.com/colaboratory/faq.html) for more info about this platform. In some cases, it might be faster to use an online Blender renderfarm.

## License
```
MIT License

Copyright (c) 2020 syn73

Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
SOFTWARE.
```